# Dissecting Goldfarb-Idnani (1983)

D. Goldfarb and A. Idnani wrote a paper, __A numerically stable dual method for solving strictly convex quadratic programs__, in Mathematical Programming 27 (1983) 1-33. 

It's a little dense, and there aren't awesomely written reference implementations out there.  Part of the goal of this notebook is to 'chainsaw' through it in order to get solid guidance for a Python implementation using NumPy/SciPy, which should aid as a reference for a reviewed implementation in Fortran 90. 

## Introduction

Directly quoting, 

>   We are concerned in this paper with the strictly convex (positive definite) quadratic programming problem 
>$$ \text{min}_x \quad f(x) = a^T x + \frac{1}{2} x^T G x $$
>
>subject to 
>
>$$ s(x) \equiv C^T x - b \geq 0 $$
>
> where $x$ and $a$ are $n$-vectors. $G$ is an $n \times n$ symmetric positive definite matrix, $C$ is an $n \times m$ matrix, $b$ is an $m$-vector, and superscript $T$ denotes the transpose.  Although the vector of variables $x$ may also be subject to equality constraints
> $$ \hat{C}\hspace{0.1em}^T x - \hat{b} = 0 $$
> we shall ignore such constraints for the moment in order to simplify our presentation. 


However, we're also going to note/paraphrase the [Wikipedia article's](https://en.wikipedia.org/wiki/Quadratic_programming#Problem_formulation) notation for the formulation of the quadratic minimization problem. 

> The quadratic programming problem with $n$ variables and $m$ constraints can be formulated as follows. Given: 
> * a real-valued $n$-dim vector $\mathbf{c}$,
> * an $n \times n$-dim real symmetric matrix $Q$,
> * an $m \times n$-dim real matrix $A$, and 
> * an $m$-dim real vector $\mathbf{b}$
>
> The objective of quadratic programming is to find an $n$-dim vector $\mathbf{x}$, that will
> $$\begin{aligned} \text{minimize} \quad & f(x) = \frac{1}{2} \mathbf{x}^T Q \mathbf{x} + \mathbf{c}^T \mathbf{x} \\ \text{subject to} \quad & \quad A \mathbf{x} \preceq \mathbf{b} \end{aligned}$$


In either of these, 
* $m$ is the number of constraints
* $n$ is the number of variables
* $\mathbf{b}$ is the constraint vector
* $A$ is a matrix representing the bound/constraint values for each $x_i$. 
* $Q \equiv G$ is the positive-definite matrix holding coefficients for the quadratic function. 
* $\mathbf{c} \equiv a$ is ???

Quoting the paper, "In this paper we present a projection type dual algorithm for solving the \[aforementioned\] QPP".  

That is, they use the Lagrangian formulation of the big problem, and recursively solve quadratic sub-problems.  Then, iterations of solving each sub-problem is involved in solving the next. 

Quoting again, 

> "In a dual method for the strictly convex QPP one must first provide a dual feasible point, that is, a primal optimal point for some subproblem of the original problem. By relaxing all of the constraints \[in the constraint problem\], the unconstrained minimum of \[ the big quadratic problem \] is such a point.  A dual algorithm then iterates until primal feasibility (i.e. dual optimality) is achieved, all the while maintaining the primal optimality of intermediate subproblems (i.e. dual feasibility)."

Paraphrasing the remainder of the paragraph: 

This can handle the positive semi-definite case for $Q$ (solving the "dual" subproblem by a "primal method" involving the original quadratic minimization problem). 

Quoting: 

> "The important observation to make is that the origin in the
space of dual variables is always dual feasible, so that no phase 1 is needed"

In other words, by recursively/iteratively solving all these dual subproblems, we end up with a feasible point that is optimal for the original problem. 


Then, Goldfarb-Idnani talk about the remaining sections of their paper. 

* __Section 2__ "Our discussion is in terms of this problem rather than in terms of the problem dual to it... We also introduce some notation in this section along with some other preliminaries.

* __Section 3__ "The dual algorithm is given ... where its validitiy and finite termination are probed." 

* __Section 4__ "A particular numerically stable and efficient implementation of the algorithm is described..."

* __Section 5__ "...we give the results of computational tests performed on randomly generated quadratic programs...."

* __Section 6__ "...the performance of our algorirthm when used in a successive quadratic programming code is described." 

> "In both of \[Sections 5 and 6\], the computational performance of the dual algorithm is compared against that of primal algorithms.

* __Section 7__ "...Comparisons between our dual algorithm and other modified simplex type dual algorithms for quadratic programming are given..."

* __Section 8__ "...we make some remarks on alternative primal approaches and implementations of our dual algorithm and comment upon its superior behavior in degenerate and 'near' degenerate situations. 

* __Appendix__ "...in which we work through an example to illustrate the various parts of the dual algorithm." 

As implementors looking at research from the 1980's, we're going to pay particular attention to Sections 2, 3, 4, and the Appendix. 